In [ ]:

import pandas as pd
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import os
import chromedriver_autoinstaller
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from concurrent.futures import ThreadPoolExecutor

opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")
opt.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])

# opt.add_argument("--lang=en")

# chromedriver_autoinstaller.install()



In [ ]:
WRITE_FLAG=True

In [ ]:

base_link="https://airtable.com/app4Y6v4bZxmLOcek/shrLGPI1xHHYhLflR/tblkRKZ9XVnWNQjI3?backgroundColor=green&layout=card&viewControls=on"             

# this websites only load few rows at a time in the DOM

root_data_folder="../data/raw/campsandclinincs/"
output_folder_name="airtable"
output_file_name="airtable"
output_file_path = os.path.join(root_data_folder,output_folder_name)

if (not os.path.exists(output_file_path)):
    os.makedirs(output_file_path)



In [ ]:
driver = webdriver.Chrome(options=opt)

In [ ]:
print(f"getting {base_link}")
driver.get(base_link)

In [ ]:
all_ids=[]

all_rows_dict=[]

In [ ]:

continue_scrolling=True


while continue_scrolling:
    try:
        
        print("scrapping all rows please wait....")
        init_length=len(all_ids)

        results=driver.find_elements(by=By.XPATH, value='.//div[@class="draggableRecord animate"]')    
    #     get all the results present in DOM
        for rzlt in results:
    #         check if row id is already scrapped or not 
            if rzlt.get_attribute("data-rowid") not in all_ids:
                each_row_dict={}

                rzlt.find_element(by=By.XPATH, value='.//div[@class="col-12 mb1 truncate strong text-size-large line-height-3"]').text
                all_details=rzlt.find_elements(by=By.XPATH, value='.//div[@class="flex-none overflow-hidden pr1"]')

    #  get all the entries in each row 
                for entry in all_details:
                    each_feild=(entry.text.split("\n"))
                    if len(each_feild)==2:
                        each_row_dict[each_feild[0]]=each_feild[-1]
    # if row is empty then
                    if len(each_feild)==1:

                        each_row_dict[each_feild[0]]=""


                all_ids.append(rzlt.get_attribute("data-rowid"))
                all_rows_dict.append(each_row_dict)

    #     scroll to load more results into DOM
        driver.execute_script("arguments[0].scrollIntoView(true);", results[-1])

        sleep(2)
    # check if new rows are loaded into dom or its end of page 
        if init_length == len(all_ids):
            continue_scrolling=False
    except Exception as e:
        print("exception caught while scraping data...")
        print(e)
        continue_scrolling=False

# driver.close()

In [ ]:
driver.close()

In [ ]:
# convert list of dicts to data frame
if WRITE_FLAG:

    df_main = pd.DataFrame(all_rows_dict)
    if df_main.shape != (0,0):
        df_main.to_csv(f"{output_file_path}/{output_file_name}.csv", index=False)
        print(f'completed,\n{df_main.shape} record found, \nFile saved successfully...')
    else:
        print('data frame is empty')
